In [1]:
import numpy as np
import itertools
import trimesh
import math
import k3d
from time import sleep

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

# !conda install -c conda-forge pyembree
# !conda install -c conda-forge igl
# !pip install Cython
# !pip install gym

In [2]:
def compute_metrics(env, agent, iter_cnt=10, max_iter=30):
    rewards, final_rewards, novp = [], [], []
    for _ in range(iter_cnt):
        state, action, mask = env.reset()
        episode_reward = 0.0
        for t in range(max_iter):
            action =  agent.act(state, mask, epsilon=0.05)
                        
            state, reward, done, info, mask = env.step(action)
            # print("REWARD: ", reward)
            # env.render(action, state)
            episode_reward += reward

            if done:
                break

        final_reward = 0
        final_reward = env.final_reward()
#         episode_reward += 1.0 / final_reward
        rewards.append(episode_reward)
        final_rewards.append(final_reward)
        novp.append(t + 1)
    return np.mean(rewards), np.mean(final_rewards), np.mean(novp)

In [12]:
from rl.environment import *

def create_env(model_path=None):    
    env = Environment(model_path=model_path,
    #                   similarity_threshold=similarity_threshold,
                      image_size=1024,
                      number_of_view_points=100)
    # env = CombiningObservationsWrapper(env)
    # env = StepPenaltyRewardWrapper(env, weight=1.0)
    # env = DepthMapWrapper(env)

    env = MeshReconstructionWrapper(env, reconstruction_depth=8, final_depth=10, scale_factor=8,
                                    do_step_reconstruction=True)
    env = VoxelGridWrapper(env, occlusion_reward=True, grid_size=64)
    env = CombiningObservationsWrapper(env)
    env = VoxelWrapper(env)
    env = StepPenaltyRewardWrapper(env)
    env = FrameStackWrapper(env, num_stack=4, lz4_compress=False)
    env = ActionMaskWrapper(env)
    return env


### DQN

In [13]:
from tqdm import tqdm_notebook as tqdm
from rl.agent import *

In [14]:
import torch
from rl.dqn import *

device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

model = torch.load("./models/abc-vdqn-occl-fix-rew/last-150000.pt")
agent = DQNAgent(env.observation_space.shape, env.action_space.n,
                 device=device)

agent.model = model.to(device)

In [15]:
%%time
models_path = "./data/1kabc/simple/val/"
result = {}
for model_path in tqdm(sorted(os.listdir(models_path))):
    env = create_env(os.path.join(models_path, model_path))
    result[model_path] = compute_metrics(env, agent, max_iter=100, iter_cnt=1)
    print(model_path, result[model_path])

/opt/conda/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/tf/NextBestViewRL/rl/agent.py:106: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  q_value = F.softmax(q_value)


0.43062321096770956 2.2900135554691294
2.720636766436839 0.1083359746434232 0.5281141045958796
0.43062321096770956 2.2111881575155965
2.641811368483306 0.0 0.5281141045958796
0.4597248028925827 0.7370880076348695
1.1968128105274523 0.2246592709984152 0.7527733755942948
0.43062321096770956 0.7373273324115913
1.167950543379301 0.0 0.7527733755942948
0.43062321096770956 0.7375957166642481
1.1682189276319577 0.0 0.7527733755942948
0.43062321096770956 0.7375485408525938
1.1681717518203034 0.0 0.7527733755942948
0.43062321096770956 0.7373727052799948
1.1679959162477043 0.0 0.7527733755942948
0.43062321096770956 0.7372904615784686
1.167913672546178 0.0 0.7527733755942948
0.43064832019283883 0.7372165942292379
1.1678649144220767 0.0 0.7527733755942948
0.43062321096770956 0.7372980392167227
1.1679212501844323 0.0 0.7527733755942948
0.43062321096770956 0.7372512719312325
1.167874482898942 0.0 0.7527733755942948
0.43062321096770956 0.7372805131774872
1.1679037241451968 0.0 0.7527733755942948
0.43

0.47338983050847455 2.6598715012128324
3.133261331721307 0.0 0.8318091451292247
0.47338983050847455 2.659527399450045
3.1329172299585193 0.0 0.8318091451292247
0.47338983050847455 2.659189145998964
3.1325789765074386 0.0 0.8318091451292247
0.47338983050847455 2.658865204450572
3.1322550349590466 0.0 0.8318091451292247
0.47338983050847455 2.6585544300408404
3.131944260549315 0.0 0.8318091451292247
0.4292372881355932 2.6610230913044806
3.090260379440074 0.013121272365805181 0.8449304174950298
0.47338983050847455 2.660408590614932
3.1337984211234065 0.0 0.8449304174950298
0.47338983050847455 2.6598705938614478
3.1332604243699222 0.0 0.8449304174950298
0.47338983050847455 2.659329524699893
3.1327193552083674 0.0 0.8449304174950298
0.47338983050847455 2.658881046548805
3.1322708770572794 0.0 0.8449304174950298
0.47338983050847455 2.6583884757061265
3.131778306214601 0.0 0.8449304174950298
0.47338983050847455 2.6579720477852278
3.1313618782937023 0.0 0.8449304174950298
0.4133050847457627 2.6

0.5910687112255648 1.6677384609705583
2.258807172196123 0.0 0.6881406659184437
0.5910687112255648 1.66742054113572
2.2584892523612847 0.0 0.6881406659184437
0.5910687112255648 1.6671976430109954
2.25826635423656 0.0 0.6881406659184437
0.5910687112255648 1.6670093184350139
2.2580780296605787 0.0 0.6881406659184437
0.5910687112255648 1.6667435134530897
2.2578122246786547 0.0 0.6881406659184437
0.5910687112255648 1.6663975096043553
2.25746622082992 0.0 0.6881406659184437
0.5910687112255648 1.6661968745195546
2.2572655857451194 0.0 0.6881406659184437
0.5910687112255648 1.6661173738337518
2.257186085059317 0.0 0.6881406659184437
0.5910687112255648 1.665981566819361
2.2570502780449258 0.0 0.6881406659184437
0.5910687112255648 1.6657812701781332
2.2568499814036977 0.0 0.6881406659184437
0.5910687112255648 1.6656283526606546
2.2566970638862194 0.0 0.6881406659184437
0.5910687112255648 1.6654813946258453
2.25655010585141 0.0 0.6881406659184437
0.5910687112255648 1.665269119076647
2.256337830302

0.39708782921452607 0.14667102798447296
0.5437588571989991 0.0 0.9073746729831011
0.39708782921452607 0.1468091947059419
0.5438970239204679 0.0 0.9073746729831011
0.39708782921452607 0.14692743664387797
0.5440152658584041 0.0 0.9073746729831011
0.39708782921452607 0.14726485122002878
0.5443526804345549 0.0 0.9073746729831011
0.39708782921452607 0.1473965040749692
0.5444843332894953 0.0 0.9073746729831011
0.39708782921452607 0.1475325633742909
0.544620392588817 0.0 0.9073746729831011
0.39708782921452607 0.14727165964083874
0.5443594888553648 0.0 0.9073746729831011
0.39708782921452607 0.1476232514776735
0.5447110806921995 0.0 0.9073746729831011
0.39708782921452607 0.14786257481422055
0.5449504040287466 0.0 0.9073746729831011
0.39708782921452607 0.14801396087747085
0.5451017900919969 0.0 0.9073746729831011
0.39708782921452607 0.14810921268968424
0.5451970419042103 0.0 0.9073746729831011
0.39708782921452607 0.14838263807057264
0.5454704672850987 0.0 0.9073746729831011
0.39711683489964034 0

...

0.34053846422460454 1.0440069156304923
1.3845453798550968 0.0 0.8388501742160279
0.34053846422460454 1.0440129987414772
1.3845514629660818 0.0 0.8388501742160279
0.34053846422460454 1.0440220234650823
1.3845604876896869 0.0 0.8388501742160279
0.34053846422460454 1.0440264917363424
1.384564955960947 0.0 0.8388501742160279
0.34053846422460454 1.0440333643525106
1.3845718285771151 0.0 0.8388501742160279
0.3636728707616021 1.043872211899449
1.4075450826610512 0.02008068952869979 0.8589308637447277
0.34053846422460454 1.0438758004976094
1.384414264722214 0.0 0.8589308637447277
0.34053846422460454 1.0438799272595771
1.3844183914841817 0.0 0.8589308637447277
0.34053846422460454 1.043874232424234
1.3844126966488386 0.0 0.8589308637447277
0.34053846422460454 1.0438710952044408
1.3844095594290453 0.0 0.8589308637447277
0.34053846422460454 1.04386555231878
1.3844040165433846 0.0 0.8589308637447277
0.34053846422460454 1.0438618797137968
1.3844003439384014 0.0 0.8589308637447277
0.34053846422460454

2.0672016907769146 0.0 0.6410350523031749
0.4935300520014512 1.573510908586155
2.0670409605876063 0.0 0.6410350523031749
0.4935300520014512 1.5733581325652881
2.0668881845667393 0.0 0.6410350523031749
0.4935300520014512 1.5732114903997791
2.0667415424012305 0.0 0.6410350523031749
0.4935300520014512 1.5730714193126476
2.0666014713140988 0.0 0.6410350523031749
0.4935300520014512 1.5729373035119518
2.066467355513403 0.0 0.6410350523031749
0.4935300520014512 1.5728086218794533
2.0663386738809044 0.0 0.6410350523031749
0.4935300520014512 1.572684598030033
2.066214650031484 0.0 0.6410350523031749
0.4935300520014512 1.5725651153063205
2.066095167307772 0.0 0.6410350523031749
0.4935300520014512 1.5724513596865464
2.0659814116879978 0.0 0.6410350523031749
0.4935300520014512 1.5723404405011538
2.0658704925026052 0.0 0.6410350523031749
0.4935300520014512 1.5722332485890405
2.0657633005904916 0.0 0.6410350523031749
0.497037126617487 0.05077934392488092
0.5478164705423679 0.34336575518443746 0.9844

0.4975062344139651 0.6547529846043727
1.1522592190183378 0.0 0.9110809894950864
0.4975062344139651 0.6548558081040394
1.1523620425180046 0.0 0.9110809894950864
0.4975062344139651 0.6532504385778403
1.1507566729918053 0.0 0.9110809894950864
0.4975062344139651 0.6532450920923905
1.1507513265063556 0.0 0.9110809894950864
0.4975062344139651 0.6532391623283252
1.1507453967422903 0.0 0.9110809894950864
0.4975062344139651 0.6532334611996327
1.1507396956135978 0.0 0.9110809894950864
0.4975062344139651 0.6532275278699239
1.150733762283889 0.0 0.9110809894950864
0.30980881130507065 0.6533714723206282
0.9631802836256989 0.0 0.9110809894950864
0.4975062344139651 0.6533669381227761
1.1508731725367412 0.0 0.9110809894950864
0.4975062344139651 0.6533416475560523
1.1508478819700174 0.0 0.9110809894950864
0.4975062344139651 0.6533164507867366
1.1508226852007017 0.0 0.9110809894950864
0.4975062344139651 0.6534004051354034
1.1509066395493686 0.0 0.9110809894950864
0.4975062344139651 0.6535261359797251
1.

0.7310333238753124 0.0 0.8726910004433279
0.4094949252297159 0.32165316892449203
0.7311480941542079 0.0 0.8726910004433279
0.4094949252297159 0.3217666348600941
0.73126156008981 0.0 0.8726910004433279
0.4094949252297159 0.32187911254262225
0.7313740377723381 0.0 0.8726910004433279
0.4094949252297159 0.32198995359778015
0.7314848788274961 0.0 0.8726910004433279
0.409464897003183 0.3220994334995651
0.7315643305027482 0.0 0.8726910004433279
0.409464897003183 0.3222082132482101
0.7316731102513931 0.0 0.8726910004433279
0.409464897003183 0.3223150697730272
0.7317799667762102 0.0 0.8726910004433279
0.409464897003183 0.3224204156002462
0.7318853126034293 0.0 0.8726910004433279
0.4429463695874122 0.3034704571130225
0.7464168267004347 0.01152652578690705 0.884217526230235
0.409464897003183 0.30313474089471776
0.7125996378979007 0.0 0.884217526230235
0.409464897003183 0.3028235700113965
0.7122884670145795 0.0 0.884217526230235
0.409464897003183 0.30253468607581446
0.7119995830789975 0.0 0.884217

0.4818008339812001 0.2910967132058851
0.7728975471870851 0.0 0.9360496786142282
0.4818008339812001 0.28990243011377054
0.7717032640949706 0.0 0.9360496786142282
0.4818008339812001 0.2903071836038118
0.7721080175850119 0.0 0.9360496786142282
0.4818008339812001 0.28947382099905167
0.7712746549802518 0.0 0.9360496786142282
0.4818008339812001 0.2899317898792074
0.7717326238604075 0.0 0.9360496786142282
0.4818008339812001 0.28929884328866334
0.7710996772698635 0.0 0.9360496786142282
0.4818008339812001 0.289779128302346
0.7715799622835461 0.0 0.9360496786142282
0.12594529648738426 0.290665325574422
0.4166106220618062 0.034099575117115144 0.9701492537313433
00020227_9d8004bda5b14a519cffb87e_000.obj (-22.531648327704545, 0.2029870851806515, 23.0)
0.49188184663536777 0.1497551930624052
0.641637039697773 0.417636934291789 0.8871114710944619
0.49188184663536777 0.14997360403797108
0.6418554506733388 0.0 0.8871114710944619
0.49188184663536777 0.1521872321058979
0.6440690787412657 0.0 0.88711147109

In [11]:
a, b, c = [], [], []
for name, res in result.items():
    a.append(res[0])
    b.append(res[1])
    c.append(res[2])
np.mean(a), np.mean(b), np.mean(c)

(-51.34526701495653, 0.2167581120874991, 51.86046511627907)

In [ ]:
def create_env(model_path=None):    
    env = Environment(model_path=model_path,
    #                   similarity_threshold=similarity_threshold,
                      image_size=512,
                      number_of_view_points=100)
    # env = CombiningObservationsWrapper(env)
    # env = StepPenaltyRewardWrapper(env, weight=1.0)
    # env = DepthMapWrapper(env)

    env = MeshReconstructionWrapper(env, reconstruction_depth=8,
                                    do_step_reconstruction=False, 
                                    scale_factor=8)

    env = VoxelGridWrapper(env, grid_size=64)
    env = CombiningObservationsWrapper(env)
    env = VoxelWrapper(env, occlusion_reward=False)
    env = StepPenaltyRewardWrapper(env, weight=0.1)
    env = FrameStackWrapper(env, num_stack=4, lz4_compress=False)
    env = ActionMaskWrapper(env)
    return env

ddict = {}
models_path = "./data/1kabc/simple/val/"

for model_name in tqdm(sorted(os.listdir(models_path))):
    env = create_env(os.path.join(models_path, model_name))

    state, _, mask = env.reset()
    novp = 0
    while True:
        action = agent.act(state, mask, epsilon=1.0)
        next_state, reward, done, _, mask = env.step(action)
        novp += 1
        if done:
            break
        ddict[model_name] = novp

In [34]:
cnt = 0
for k, v in ddict.items():
    if v > 20:
        cnt += 1

In [35]:
cnt

11

In [38]:
for k, v in ddict.items():
    if v > 20:
        os.remove(os.path.join("./data/1kabc/simple/val", k))

(-10.043238681257929, 0.09227791846438424, 22.22222222222222)

### Random

In [ ]:
%%time
random_agent_func = lambda s : env.action_space.sample()
models_path = "./data/10abc/"
result = {}
for model_path in os.listdir(models_path):
    env = create_env(os.path.join(models_path, model_path))
    result[model_path] = compute_metrics(env, random_agent_func, max_iter=30)
    print(model_path, result[model_path])

00731313_58fc1a1bb4b5be10cb401503_003.obj (-2.5757095519377606, 0.25795545337154036, 4.3)
00942098_bd039b3a4a4efa75e86b7350_000.obj (-3.0503391472868215, 0.10782423963311767, 4.9)


In [37]:
df = pd.read_csv("./abc_100_train.csv")

model_paths = []
for ix, row in df.iterrows():
    if row["Greedy_novp"] < 10:
        model_paths.append(row["Name"])

In [40]:
for path in model_paths:
    try:
        os.rename(path, os.path.join("./data/1kabc/simple/train", os.path.basename(path)))
    except:
        pass